## **Import Libraries**

In [28]:
import pandas as pd 
import os
import yaml
from pathlib import Path

## **Load configs**

In [20]:
project_root = Path.cwd().parent  
config_path = project_root / "config" / "config.yaml"
CFG = yaml.safe_load(open(config_path))
RAW = project_root / CFG["paths"]["data_raw"]
SEP = CFG.get("io", {}).get("csv_sep", ",")

## **Read data**

In [22]:
assert RAW.exists(), f"Missing file {RAW}"

df = pd.read_csv(RAW, sep=SEP)
print(df.shape)

(1599, 12)


## **Contract check (required columns)**

In [ ]:
SCHEMA = yaml.safe_load(open(project_root /"schema.yaml"))
required = set(SCHEMA.get("required", [])) | {SCHEMA.get("target", "quality")}
missing = required - set(df.columns)
print(missing)

set()